In [1]:
from rich import print
import os
import pathlib
import random
import sys
from asyncio.log import logger
from collections import defaultdict
from dataclasses import dataclass
from math import floor
from typing import Any, Dict, List, Optional, Tuple, Union

from torchtyping import TensorType

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import tqdm
from hydra_zen import builds, instantiate
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, RandomCrop, Resize, ToTensor
from traitlets import Int
from capit.core.data.config import ImageShape, ModalityConfig
from capit.core.utils.storage import load_json, save_json

from capit.decorators import configurable
from capit.utils import get_logger
from torch.utils.data.dataloader import default_collate

logger = get_logger(__name__)

os.environ["HYDRA_FULL_ERROR"] = "1"



/opt/conda/envs/main/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from capit.core.data.datasets import get_image_transforms_instait, \
        InstagramImageTextMultiModalDatasePyArrow, ChallengeSamplesSourceTypes, \
        SplitType 

root_filepath = pathlib.Path("/data/")
dataset_config = InstagramImageTextMultiModalDatasePyArrow.build_config(
    populate_full_signature=True
)
dataset_config_instance = dataset_config(dataset_dir=root_filepath)
train_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.TRAIN,
)

val_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.VAL,
)

test_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.TEST,
)

In [4]:
import tqdm
train_usernames = train_dataset.set_usernames
val_usernames = val_dataset.set_usernames
test_usernames = test_dataset.set_usernames
with tqdm.tqdm(total=len(train_usernames)) as pbar:
    for username in train_usernames:
        assert username not in val_usernames
        assert username not in test_usernames
        pbar.update(1)

with tqdm.tqdm(total=len(val_usernames)) as pbar:
    for username in val_usernames:
        assert username not in train_usernames
        assert username not in test_usernames
        pbar.update(1)

with tqdm.tqdm(total=len(test_usernames)) as pbar:
    for username in test_usernames:
        assert username not in train_usernames
        assert username not in val_usernames
        pbar.update(1)


100%|██████████| 3065/3065 [00:01<00:00, 2795.60it/s]
